In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import os
from cleantext import clean
from pandas import DataFrame
import random as rnd

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
import unittest

Vérification de la liste des fichiers, ce notebook nécessite au moins la présence de Data_train.csv Data_train_2.csv Label_train.csv Label_train_2.csv

In [3]:
arr = os.listdir('.')
print(arr)

['Label_train_2.csv', '.DS_Store', 'Creation_matrice_Creation.ipynb', 'requirements.txt', 'FinalDBelon.csv', 'Data_train.csv', 'DataNONEL_clean.csv', 'Clean_Tweets_EM.ipynb', 'Creation_matrice_Detection.ipynb', 'README.md', 'DataBase_temporary_creation.csv', 'Tweets_Not_ElonMusk.csv', 'DataBase.csv', '.gitignore', 'Dialogue.ipynb', 'Data_train_2.csv', 'DataElon_clean.csv', '.ipynb_checkpoints', 'Label_train.csv', 'Clean_Tweets_NONEM.ipynb', '.git', 'TweetsElonMusk.csv']


Test si la version de tensor flow utilisée est bien 2.0, ne renvoie rien après son exécution si la bonne version est utilisée

In [4]:
assert hasattr(tf, 'function')

 ### Importation des datasets d'entrainement.
\
La partie Detection a pour dataset d'entrainement Data_train et ses labels Label_train.
\
La partie Creation a pour dataset d'entrainement Data_train_2 et ses labels Label_train_2.

In [5]:
Data_train = pd.read_csv("Data_train.csv")
Label_train = pd.read_csv("Label_train.csv")

In [6]:
Data_train_2 = pd.read_csv('Data_train_2.csv')
Label_train_2 = pd.read_csv("Label_train_2.csv")

 ### Construction du modele utilisé pour les deux réseaux de neurones

In [7]:
def Construction_modele(Data_train, Label_train, affichage=1):
    #Construction couches de neurones
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))
    #Mise au bon format des donnees pour le modele
    Tweet_1 = Data_train.values
    Tweet_2 = Tweet_1.astype(dtype='float32')
    Label_1 = Label_train.values
    Label_2 = Label_1[:,0].astype(dtype='uint8')
    #Ajustement modele et choix de metrique
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    #Entrainement du modele
    model.fit(Tweet_2, Label_2, epochs=18, verbose=affichage)
    return model

 ### Fonction de vectorisation de tweet
 Prend un tweet en argument sous la forme d'une chaîne de caractères et renvoie son vecteur sous forme d'une database dans l'espace vectoriel de plongement utilisé par l'argument Data_train

In [8]:
def Vectorisation(Data_train, tweet):  
    Vecteurs = Data_train.columns
    row = []
    
    dictionnary = dict()
    for mot in Vecteurs:
        dictionnary[mot] = 0
    
    Mot_tweet = tweet.split(" ")
    
    for index, mot in enumerate(Mot_tweet):
            if mot in Vecteurs:
                dictionnary[mot] = index + 1
                
    row.append(dictionnary)
    Data_out = DataFrame(row)
    
    if Data_out.loc[0].max() != 0:
        Data_out.loc[0] = Data_out.loc[0] / Data_out.loc[0].max()
    
    return Data_out
    
    
    

 ### Fonction de normalisation d'un vecteur 
Le vecteur est normalisé par rapport à la norme infinie $||vecteur||_{\infty} = 1$ ou $0$

In [9]:
def normalisation(x):
    if x.max() != 0:
        x = x / x.max()
    return x

 ### Fonction de vectorisation d'une liste de tweets
 Semblable à vectorisation

In [10]:
def Vectorisation_liste(Data_train, liste_tweet, func_norma = normalisation):   
    Vecteurs = Data_train.columns
    
    row = []
    
    dictionnary = dict()
    for mot in Vecteurs:
        dictionnary[mot] = 0
    
    for tweet in liste_tweet:
        
        Mot_tweet = tweet.split(" ")
        dictionnary2 = dictionnary.copy()
        for index, mot in enumerate(Mot_tweet):
            if mot in Vecteurs:
                dictionnary2[mot] = index + 1
                
        row.append(dictionnary2)
        
    Data_out = DataFrame(row)
    Data_out = Data_out.apply(normalisation, axis=1)
        
    return Data_out

 ### Fonction de détection d'un tweet
La fonction de détection prend en argument un tweet sous la forme d'une chaîne de caractères. Le réseau de neurones entrainé passé en argument renvoie son évaluation du tweet, sous la forme d'un tuple :
\
\
("Elon Musk detecte a x%", 1)
\
("Indetermine", 2)
\
("Pas Elon Musk a x%", 0)
\
\
Si la probabilité selon le réseau que le tweet appartienne à Elon Musk est supérieure à 60%, le tweet sera considéré comme appartenant à Elon Musk, si la probabilité que le tweet appartienne à Elon Musk est inférieure à 40%, le tweet sera considéré comme n'appartenant pas à Elon Musk. Si la probabilité est comprise entre 40% et 60%, le tweet sera considéré comme indéterminé.


In [11]:
def detection(tweet_sentence, model_neurone, Data_train):
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    vecteur = vecteur_phrase.values
    vecteur = vecteur.astype(dtype='float32')
    if vecteur.sum() == 0:
        char = "Indetermine"
        return (char, 2)
    Prediction = model_neurone.predict(vecteur)
    if Prediction[0, 1] >= 0.6:
        char = "Elon Musk detecte a " + str(Prediction[0, 1]) + "%"
        return (char, 1)
    elif Prediction[0,1] >= 0.4:
        char = "Indetermine"
        return (char, 2)
    else :
        char = "Pas Elon Musk a " + str(Prediction[0, 0]) + "%"
        return (char, 0)

 ### Fonction de détection d'un tweet dans une forme brut
 Renvoie l'évaluation d'un tweet par le réseau de neurones entrainé de manière brute i.e. la probabilité que ce soit un tweet de Elon Musk

In [12]:
def detection_creation(tweet_sentence, model_neurone, Data_train):
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    vecteur = vecteur_phrase.values
    vecteur = vecteur.astype(dtype='float32')
    if vecteur.sum() == 0:
        return 0
    Prediction = model_neurone.predict(vecteur)
    return Prediction[0, 1]

Exemple:

In [20]:
#detection_creation("Spacex is the futur of humanity, and space will be our home", reseau_neurone_creation, Data_train_2)

0.92967653

 ### Fonction de détection et d'apprentissage
La fonction détecte et rend une estimation du tweet proposé et l'ajoute à son entrainement




In [14]:
def detection_apprentissage(tweet_sentence, model_neurone, Data_train, Label_train, val=0):
    #estimation
    (char, val) = detection(tweet_sentence, model_neurone, Data_train)
    #re entrainement du reseau de neurones
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    Data_train = pd.concat([Data_train, vecteur_phrase], ignore_index=True)
    Data_train.reset_index(drop=True, inplace=True)
    label = DataFrame([{'label':val}])
    Label_train = pd.concat([Label_train, label], ignore_index=True)
    Label_train.reset_index(drop=True, inplace=True)
    model = Construction_modele(Data_train, Label_train, affichage=0)
    return ((char, val), model)

### Fonction de détection et d'apprentissage
La fonction de détection et d'apprentissage d'une liste prend en argument une liste de tweet sous la forme ["tweet1", "tweet2"…] et les labels associés sous la forme d'une liste [0,0,1…]. La fonction renvoie une liste [model_neurone, Data, Label, tweet_liste, valeur]
\
$ \textbf{model_neurone} $ est le réseau de neurones réentrainé avec la liste de tweet fournie et aux labels indiqués en entrée.
\
$ \textbf{Data} $ et $ \textbf{Label} $ sont les nouveaux datasets d'entrainement du réseau, qui sont ceux passés en arguments et concaténés avec les nouveaux
\
$ \textbf{tweet_liste} $ est la liste de tweets initiaux passée en argument
\
$ \textbf{valeur} $ est une liste des estimations évaluées avant ré-apprentissage du réseau de neurones 

In [15]:
def detection_apprentissage_liste(tweet_liste, model_neurone, Data_train, Label_train, val_list):
    Estimat = []
    Data_adding = Vectorisation_liste(Data_train, tweet_liste)
    Data = Data_train.copy()
    
    Label_adding = pd.DataFrame(val_list, columns=['label'])
    Label = Label_train.copy()
    
    Data_add = Data_adding.values
    Data_add = Data_add.astype(dtype='float32')
    
    valued = model_neurone.predict(Data_add)
    
    Data = pd.concat([Data, Data_adding], ignore_index = True)
    Label = pd.concat([Label, Label_adding], ignore_index = True)
    
    model_neurone = Construction_modele(Data, Label, affichage=0)
    
    valeur = (valued[:,1] > 0.5).astype(dtype='int').tolist()

    
    return [model_neurone, Data, Label, tweet_liste, valeur]
    
    

### Fonction de création d'un tweet
La fonction a pour but de créer un tweet d'une longueur attendue, et renvoie un tweet que le réseau de neurones juge soit acceptable (à une estimation supérieure à 0.9) soit a essayé d'atteindre ce niveau (estimation supérieure à 0.9) 100 fois (sans réussir).



In [16]:
def Creation_tweet(nombre_mots_total, phrase, Data_train_2, model_neurone): 
    phrase = phrase.lower()
    phrase = clean(phrase, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    liste_mot =  phrase.split(" ")
    if nombre_mots_total < len(liste_mot):
        print("Erreur liste mot plus longue que le tweet prevu, longueur du tweet pris par defaut au nombre de mots")
        nombre_mots_total = len(liste_mot)
    nombre_mot_rest = nombre_mots_total - len(liste_mot)
    
    liste_tweet = liste_mot + rnd.sample(list(Data_train_2.columns), nombre_mot_rest, counts=None)
    tweet_pertinent = ' '.join(liste_tweet)
    pertinence = detection_creation(tweet_pertinent, model_neurone, Data_train_2)
    
    for iteration in range(100):
        if pertinence > 0.9:
            tweet_pertinent = clean(tweet_pertinent, no_punct=True)
            return tweet_pertinent
        liste = liste_mot + rnd.sample(list(Data_train_2.columns), nombre_mot_rest, counts=None)
        rnd.shuffle(liste)
        current_tweet = ' '.join(liste)
        current_pertinence = detection_creation(current_tweet, model_neurone, Data_train_2)
        if current_pertinence > pertinence:
            pertinence = current_pertinence
            tweet_pertinent = current_tweet
    current = clean(tweet_pertinent, no_punct=True)
    return current
        
    

# Initialisation des réseaux de neurones avant le dialogue

In [17]:
reseau_neurone_detection = Construction_modele(Data_train, Label_train)

Epoch 1/18
348/348 [==============================] - 1s 2ms/step - loss: 0.6869 - accuracy: 0.5757
Epoch 2/18
348/348 [==============================] - 1s 2ms/step - loss: 0.6697 - accuracy: 0.6813
Epoch 3/18
348/348 [==============================] - 1s 2ms/step - loss: 0.6429 - accuracy: 0.7255
Epoch 4/18
348/348 [==============================] - 1s 1ms/step - loss: 0.5986 - accuracy: 0.7565
Epoch 5/18
348/348 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7803
Epoch 6/18
348/348 [==============================] - 1s 1ms/step - loss: 0.4776 - accuracy: 0.7987
Epoch 7/18
348/348 [==============================] - 1s 2ms/step - loss: 0.4279 - accuracy: 0.8186
Epoch 8/18
348/348 [==============================] - 1s 1ms/step - loss: 0.3908 - accuracy: 0.8339
Epoch 9/18
348/348 [==============================] - 1s 1ms/step - loss: 0.3633 - accuracy: 0.8440
Epoch 10/18
348/348 [==============================] - 1s 1ms/step - loss: 0.3419 - accuracy: 0.8518

In [18]:
reseau_neurone_creation = Construction_modele(Data_train_2, Label_train_2)

Epoch 1/18
370/370 [==============================] - 1s 2ms/step - loss: 0.6883 - accuracy: 0.5476
Epoch 2/18
370/370 [==============================] - 1s 2ms/step - loss: 0.6788 - accuracy: 0.6109
Epoch 3/18
370/370 [==============================] - 1s 2ms/step - loss: 0.6650 - accuracy: 0.6534
Epoch 4/18
370/370 [==============================] - 1s 2ms/step - loss: 0.6440 - accuracy: 0.6867
Epoch 5/18
370/370 [==============================] - 1s 1ms/step - loss: 0.6152 - accuracy: 0.7113
Epoch 6/18
370/370 [==============================] - 1s 1ms/step - loss: 0.5809 - accuracy: 0.7318
Epoch 7/18
370/370 [==============================] - 1s 2ms/step - loss: 0.5467 - accuracy: 0.7505
Epoch 8/18
370/370 [==============================] - 1s 1ms/step - loss: 0.5164 - accuracy: 0.7661
Epoch 9/18
370/370 [==============================] - 1s 2ms/step - loss: 0.4902 - accuracy: 0.7794
Epoch 10/18
370/370 [==============================] - 1s 2ms/step - loss: 0.4674 - accuracy: 0.7928

# Début du dialogue

In [21]:
Data_train_Detection = Data_train.copy()
Data_train_Creation = Data_train_2.copy()

Label_train_Detection = Label_train.copy()
Label_train_Creation = Label_train_2.copy()

In [22]:
def dialogue(iteration, nombre_mot ,reseau_neurone_detection, reseau_neurone_creation, Data_train, Data_train_2, Label_train, Label_train_2):
    nombre_mot += 1
    evolution = []
    for i in range(iteration):
        
        #Creation parle en premier et propose 30 tweets, qu il juge pertinent, a Detection
        print('Tweet propose par creation a la ', i , 'eme iteration : \n')
        tweet_liste = []
        valeur = [0]*30 
        for j in range(30):
            
            tweet_proposal = Creation_tweet(nombre_mot, '', Data_train_2, reseau_neurone_creation)
            print(tweet_proposal)
            tweet_liste.append(tweet_proposal)
        print('\n')
        
        #Detection recoit ces 30 tweets, les evalues, les appreds en tenant compte qu ils sont faux, et retourne a creation ce qu il en pensait avant d apprendre qu ils etaient faux
        
        [reseau_neurone_detection, Data_train, Label_train, tweet_liste, evaluation] = detection_apprentissage_liste(tweet_liste, reseau_neurone_detection, Data_train, Label_train, valeur)
        
        #Affichage de ce que globalement detection en pensait
        
        print('Sur ces tweets propose detection en a trouve :', sum(evaluation), ' correct selon lui')
        
        evolution.append(sum(evaluation))
        
        #Creation apprend de ses tweets en les labelisant suivant ce que detection en avait pense a priori
        
        [reseau_neurone_creation, Data_train_2, Label_train_2, _, _] = detection_apprentissage_liste(tweet_liste, reseau_neurone_creation, Data_train_2, Label_train_2, evaluation)
        
        print('\n \n')
        
    return [evolution, reseau_neurone_detection, reseau_neurone_creation, Data_train, Data_train_2, Label_train, Label_train_2]
            
            
            
            
            

In [23]:
dialogue_summary = dialogue(1, 7, reseau_neurone_detection, reseau_neurone_creation, Data_train_Detection, Data_train_Creation, Label_train_Detection, Label_train_Creation)

Tweet propose par creation a la  0 eme iteration : 

so st short tube tune > both
carpet line obama found free slow release
posted really final exactly youve die way
stop using matters mph also hell complete
kids covered but media news water ground
jobs solar clear nasa tiny model cute
awesome inside fair using navy definitely finally
grateful wonderful turn tesla matters video btw
met internet worked thursday record home california
yes future orbit here autopilot ship use
nasa all art starting birthday wrong definitely
forward want nasa built up guy landing
gets pressure photo years model supercharger rate
camera going price thank free landing into
turn booster hw boring build come data
mean congratulations join thrust end drive matters
york vote barbados jobs hyperloop car will
rio sounds awesome using fully hair course
spacex few person second via love station
until friend step quality about exciting landing
cheek free makes month really matter very
foundation thats via station thin

### Tests unitaires 
Tests unitaires de certaines fonctions clés

Ce test unitaire à pour but de vérifier que la fonction Vectorisation renvoie un bien une DataFrame lorsqu'elle prend une phrase quelconque en argument. Le test verifiera aussi que les colonnes du DataFrame crée corresponde avec celui de Data_train passé en argument.


In [26]:
import uuid

In [30]:
def test_unitaire_vectorisation():
    tweet = str(uuid.uuid4())
    tweet = tweet.replace('-', ' ')
    test_list = np.random.randint(10, size=(1,3))
    Data_testunitaire = pd.DataFrame(test_list, columns=['column1', 'column2', 'column3'])
    for i in range(10):
        vectorisation = Vectorisation(Data_testunitaire, tweet)
        if vectorisation.shape != Data_testunitaire.shape:
            return False
        if not (vectorisation.columns == Data_testunitaire.columns).all():
            return False
    return True

In [31]:
test_unitaire_vectorisation()

True